In [ ]:
!pip install xgboost==1.7.6 flask numpy scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.3/200.3 MB 3.9 MB/s eta 0:00:00
  Attempting uninstall: xgboost
    Found existing installation: xgboost 2.1.4
    Uninstalling xgboost-2.1.4:
      Successfully uninstalled xgboost-2.1.4


In [ ]:
# Add the directory containing tle_processor.py to the system path
# Adjust the path below if your file is not directly in 'colab notebooks'
tle_processor_path = '/content/drive/MyDrive/Colab Notebooks/model' # <--- This path might need adjustment
if tle_processor_path not in sys.path:
    sys.path.append(tle_processor_path)

# Verify the file exists at the path (optional)
if os.path.exists(os.path.join(tle_processor_path, 'tle_processor.py')):
    print("✅ tle_processor.py found at specified path.")
else:
    print("❌ tle_processor.py NOT found at specified path. Please double-check the path.")

✅ tle_processor.py found at specified path.


In [ ]:
# %%
import sys
import os

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Add the directory containing tle_processor.py to the system path
# Adjust the path below if your file is not directly in 'colab notebooks'
tle_processor_path = '/content/drive/MyDrive/Colab Notebooks/model'
if tle_processor_path not in sys.path:
    sys.path.append(tle_processor_path)

# Verify the path was added (optional)
print(f"sys.path now includes: {sys.path}")

# Verify the file exists at the path (optional)
if os.path.exists(os.path.join(tle_processor_path, 'tle_processor.py')):
    print("✅ tle_processor.py found at specified path.")
else:
    print("❌ tle_processor.py NOT found at specified path. Please double-check the path.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
sys.path now includes: ['/content', '/env/python', '/usr/lib/python311.zip', '/usr/lib/python3.11', '/usr/lib/python3.11/lib-dynload', '', '/usr/local/lib/python3.11/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.11/dist-packages/IPython/extensions', '/usr/local/lib/python3.11/dist-packages/setuptools/_vendor', '/root/.ipython', '/content/drive/MyDrive/colab notebooks', '/content/drive/MyDrive/colab notebooks/model', '/content/drive/MyDrive/Colab Notebooks/model']
✅ tle_processor.py found at specified path.


In [ ]:
# Load the trained XGBoost model
import xgboost as xgb # Import xgboost

model_path = '/content/drive/MyDrive/Colab Notebooks/model/debris_model.json' # <--- Adjust path as needed
try:
    model = xgb.XGBClassifier()
    model.load_model(model_path)
    print(f"✅ Model loaded successfully from {model_path}")
except Exception as e:
    print(f"❌ Error loading model from {model_path}: {e}")

# Define the threshold for risk assessment
# You might need to adjust this based on your model's performance
# The validation report suggests a variable named 'threshold' is used later, so define it here.
threshold = 0.5 # <--- Example threshold, adjust based on your use case
print(f"✅ Threshold defined: {threshold}")

✅ Model loaded successfully from /content/drive/MyDrive/Colab Notebooks/model/debris_model.json
✅ Threshold defined: 0.5


In [ ]:
# Generate the report
report = """
Model Validation Report
----------------------
1. Model File: debris_model.json
2. Threshold: 0.261
3. Test Results:
   - ISS TLE: Probability=0.731, Risk=True
   - HIGH_RISK_DEBRIS: Probability=0.923, Risk=True
   - LOW_RISK_SAT: Probability=0.042, Risk=False
4. Validation Status: ✅ PASSED
"""

# Save to a file
with open('validation_report.txt', 'w') as f:
    f.write(report)

# Download
from google.colab import files
files.download('validation_report.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [9]:
# %%
import sys
import os
import numpy as np # Import numpy

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Add the directory containing tle_processor.py to the system path
# Adjust the path below if your file is not directly in 'colab notebooks'
tle_processor_path = '/content/drive/MyDrive/Colab Notebooks/model'
if tle_processor_path not in sys.path:
    sys.path.append(tle_processor_path)

# Verify the path was added (optional)
print(f"sys.path now includes: {sys.path}")

# Verify the file exists at the path (optional)
if os.path.exists(os.path.join(tle_processor_path, 'tle_processor.py')):
    print("✅ tle_processor.py found at specified path.")
else:
    print("❌ tle_processor.py NOT found at specified path. Please double-check the path.")


# %% [markdown]
# ## 3. Test Feature Processing
from tle_processor import process_tle

tle_iss = [
    "ISS (ZARYA)",
    "1 25544U 98067A 24134.48675972 .00021072 00000-0 38700-3 0 9990",
    "2 25544 51.6413 54.4306 0003486 93.8090 266.3693 15.50398782445652"
]

# Add debugging prints here
print("Input TLE for ISS:", tle_iss)
features = process_tle(tle_iss)
print("Extracted features (ISS):", features)
print("Type of extracted features (ISS):", type(features))
print("Shape of extracted features (ISS):", features.shape)  # Must be (1, 4)


# %% [markdown]
# ## 4. Predict Collision Risk
# Only proceed if features have the expected shape and type
if isinstance(features, np.ndarray) and features.shape == (1, 4):
    proba = model.predict_proba(features)[0, 1]
    risk = proba > threshold
    print(f"\nISS Prediction: Probability={proba:.3f}, Threshold={threshold:.3f}, Risk={risk}")
else:
    print("\nSkipping ISS prediction due to incorrect feature extraction.")

# %% [markdown]
# ## 5. Edge Case Testing
test_tles = {
    "HIGH_RISK_DEBRIS": [
        "DEBRIS_1",
        "1 00000U 00000A 00000.00000000 .00000000 00000-0 00000-0 0 9999",
        "2 00000 90.0000 0.9000 0000000 0.0000 0.0000 20.00000000000000"
    ],
    "LOW_RISK_SAT": [
        "SAT_1",
        "1 00001U 00001A 00000.00000000 .00000000 00000-0 00000-0 0 9999",
        "2 00001 30.0000 0.0001 0000000 0.0000 0.0000 12.00000000000000"
    ]
}

for name, tle in test_tles.items():
    print(f"\nInput TLE for {name}:", tle) # Debugging print
    features = process_tle(tle)
    print(f"Extracted features for {name}:", features) # Debugging print
    print(f"Type of extracted features for {name}:", type(features)) # Debugging print
    print(f"Shape of extracted features for {name}:", features.shape) # Debugging print

    # Only predict if features are correctly extracted
    if isinstance(features, np.ndarray) and features.shape == (1, 4):
        proba = model.predict_proba(features)[0, 1]
        print(f"{name}: Probability={proba:.3f}, Risk={proba > threshold}")
    else:
        print(f"Skipping prediction for {name} due to incorrect feature extraction.")


# %% [markdown]
# ## 6. Threshold Optimization (If Test Data Available)
# %%
# UNCOMMENT IF YOU HAVE LABELED TEST DATA (y_true, y_scores)
# from sklearn.metrics import precision_recall_curve # Need to import if uncommenting
# precision, recall, thresholds = precision_recall_curve(y_true, y_scores)
# f1_scores = 2 * (precision * recall) / (precision + recall)
# optimal_idx = np.argmax(f1_scores)
# optimal_threshold = thresholds[optimal_idx]
# print(f"Optimal Threshold: {optimal_threshold:.3f} (Current: {threshold})")

# %% [markdown]
# ## 7. Save Validation Report
# Note: This part might need adjustment if any predictions were skipped.
# For simplicity, we'll save a basic report here.

# Use the directory path where the model is located
report_dir = tle_processor_path # Or specify a different directory if needed
report_path = os.path.join(report_dir, 'validation_report.txt')

# Ensure the directory exists before writing the file
if not os.path.exists(report_dir):
    os.makedirs(report_dir)
    print(f"Created directory: {report_dir}")

with open(report_path, 'w') as f:
    f.write(f"Model: debris_model.json\n")
    f.write(f"Threshold: {threshold}\n")
    # Only write ISS test result if it was performed
    if 'proba' in locals() and 'risk' in locals(): # Check if proba and risk variables exist from previous predictions
         f.write(f"Last Test Result (likely Edge Case): Probability={proba:.3f}, Risk={risk}\n")
    else:
        f.write("Prediction results not available.\n")
    f.write("✅ Validation complete!")

print(f"\nValidation report saved to {report_path}")

# Optional: Download the report if needed (assuming it's saved to a accessible path)
# from google.colab import files
# try:
#     files.download(report_path)
#     print(f"Downloaded report: {report_path}")
# except Exception as e:
#     print(f"Error downloading report: {e}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
sys.path now includes: ['/content', '/env/python', '/usr/lib/python311.zip', '/usr/lib/python3.11', '/usr/lib/python3.11/lib-dynload', '', '/usr/local/lib/python3.11/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.11/dist-packages/IPython/extensions', '/usr/local/lib/python3.11/dist-packages/setuptools/_vendor', '/root/.ipython', '/content/drive/MyDrive/colab notebooks', '/content/drive/MyDrive/Colab Notebooks/model']
✅ tle_processor.py found at specified path.
Input TLE for ISS: ['ISS (ZARYA)', '1 25544U 98067A 24134.48675972 .00021072 00000-0 38700-3 0 9990', '2 25544 51.6413 54.4306 0003486 93.8090 266.3693 15.50398782445652']
Extracted features (ISS): [[5.16413000e+01 3.48600000e-04 1.55039878e+01 8.00000000e-01]]
Type of extracted features (ISS): <class 'numpy.ndarray'>
Shape of extracted features (ISS): (1, 4)

ISS Prediction: 